%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

Computational drug discovery: part 2

Exploratory Data Analysis

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

In [ ]:
# Install conda and rdkit
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2021-08-26 16:07:02--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.1’

Miniconda3-py37_4.8 100%[===================>]  81.12M   129MB/s    in 0.6s    

2021-08-26 16:07:03 (129 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh.1’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ | / - \ | / - \ | / - failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | / - \ | / - \ | / - failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: - \ | / - \ | / 

In [ ]:
# Import libraries
import pandas as pd

In [ ]:
! wget -O genital_herpes_03_bioactivity_data_curated.csv https://raw.githubusercontent.com/AnVales/Drug-discovery/main/genital_herpes_03_bioactivity_data_curated.csv

--2021-08-26 16:25:25--  https://raw.githubusercontent.com/AnVales/Drug-discovery/main/genital_herpes_03_bioactivity_data_curated.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21267 (21K) [text/plain]
Saving to: ‘genital_herpes_03_bioactivity_data_curated.csv’

genital_herpes_03_b 100%[===================>]  20.77K  --.-KB/s    in 0.001s  

2021-08-26 16:25:25 (13.8 MB/s) - ‘genital_herpes_03_bioactivity_data_curated.csv’ saved [21267/21267]



In [ ]:
bioactivity_data = pd.read_csv('genital_herpes_03_bioactivity_data_curated.csv')
bioactivity_data

,molecule_chembl_id,canonical_smiles,standard_value,bioactivity
0,CHEMBL122071,CC(=O)O[C@@H]1CC(=O)N1C(=O)NC(C)C,100000.0,inactive
1,CHEMBL124107,CCCCNC(=O)N1C(=O)C[C@H]1OC(C)=O,68000.0,inactive
2,CHEMBL120853,C[C@@H]1C(=O)N(C(=O)NCc2ccccc2)[C@@H]1Oc1ccc(C...,100000.0,inactive
3,CHEMBL122296,CC(=O)O[C@@H]1[C@@H](C)C(=O)N1C(=O)NCc1ccccc1,6500.0,intermediate
4,CHEMBL333734,CC(=O)O[C@@H]1[C@H](C)C(=O)N1C(=O)NCc1ccccc1,27000.0,inactive
...,...,...,...,...
230,CHEMBL107357,Cc1csc2nc(Cc3ccc(C(=O)c4ccc(CO)cc4)cc3)oc(=O)c12,940.0,intermediate
231,CHEMBL322676,Cc1csc2nc(Cc3cccc(NC(=O)c4cccc(NO)c4)c3)oc(=O)c12,45.0,intermediate
232,CHEMBL104662,Cc1csc2nc(Cc3ccc(C(=O)c4ccccc4)cc3)oc(=O)c12,800.0,inactive
233,CHEMBL106921,Cc1csc2nc(Cc3cccc(NC(=O)c4ccccc4)c3)oc(=O)c12,1500.0,active


In [ ]:
# Column rearrangement
bioactivity_data_no_smiles = bioactivity_data.drop(columns='canonical_smiles')
bioactivity_data_no_smiles

,molecule_chembl_id,standard_value,bioactivity
0,CHEMBL122071,100000.0,inactive
1,CHEMBL124107,68000.0,inactive
2,CHEMBL120853,100000.0,inactive
3,CHEMBL122296,6500.0,intermediate
4,CHEMBL333734,27000.0,inactive
...,...,...,...
230,CHEMBL107357,940.0,intermediate
231,CHEMBL322676,45.0,intermediate
232,CHEMBL104662,800.0,inactive
233,CHEMBL106921,1500.0,active


In [ ]:
# Obtain the smiles
smiles = []

for i in bioactivity_data.canonical_smiles.tolist():
  cpd = str(i).split('.')
  cpd_longest = max(cpd, key = len)
  smiles.append(cpd_longest)

smiles = pd.Series(smiles, name = 'canonical_smiles')
smiles

0                      CC(=O)O[C@@H]1CC(=O)N1C(=O)NC(C)C
1                        CCCCNC(=O)N1C(=O)C[C@H]1OC(C)=O
2      C[C@@H]1C(=O)N(C(=O)NCc2ccccc2)[C@@H]1Oc1ccc(C...
3          CC(=O)O[C@@H]1[C@@H](C)C(=O)N1C(=O)NCc1ccccc1
4           CC(=O)O[C@@H]1[C@H](C)C(=O)N1C(=O)NCc1ccccc1
                             ...                        
230     Cc1csc2nc(Cc3ccc(C(=O)c4ccc(CO)cc4)cc3)oc(=O)c12
231    Cc1csc2nc(Cc3cccc(NC(=O)c4cccc(NO)c4)c3)oc(=O)c12
232         Cc1csc2nc(Cc3ccc(C(=O)c4ccccc4)cc3)oc(=O)c12
233        Cc1csc2nc(Cc3cccc(NC(=O)c4ccccc4)c3)oc(=O)c12
234    Cc1csc2nc(Cc3cccc(NC(=O)c4ccc(NN)cc4)c3)oc(=O)c12
Name: canonical_smiles, Length: 235, dtype: object

In [ ]:
bioactivity_data_smiles = pd.concat([bioactivity_data_no_smiles,smiles], axis=1)
bioactivity_data_smiles

,molecule_chembl_id,standard_value,bioactivity,canonical_smiles
0,CHEMBL122071,100000.0,inactive,CC(=O)O[C@@H]1CC(=O)N1C(=O)NC(C)C
1,CHEMBL124107,68000.0,inactive,CCCCNC(=O)N1C(=O)C[C@H]1OC(C)=O
2,CHEMBL120853,100000.0,inactive,C[C@@H]1C(=O)N(C(=O)NCc2ccccc2)[C@@H]1Oc1ccc(C...
3,CHEMBL122296,6500.0,intermediate,CC(=O)O[C@@H]1[C@@H](C)C(=O)N1C(=O)NCc1ccccc1
4,CHEMBL333734,27000.0,inactive,CC(=O)O[C@@H]1[C@H](C)C(=O)N1C(=O)NCc1ccccc1
...,...,...,...,...
230,CHEMBL107357,940.0,intermediate,Cc1csc2nc(Cc3ccc(C(=O)c4ccc(CO)cc4)cc3)oc(=O)c12
231,CHEMBL322676,45.0,intermediate,Cc1csc2nc(Cc3cccc(NC(=O)c4cccc(NO)c4)c3)oc(=O)c12
232,CHEMBL104662,800.0,inactive,Cc1csc2nc(Cc3ccc(C(=O)c4ccccc4)cc3)oc(=O)c12
233,CHEMBL106921,1500.0,active,Cc1csc2nc(Cc3cccc(NC(=O)c4ccccc4)c3)oc(=O)c12


In [ ]:
# Rule-of-Five or Lipinski's Rule.
# Evaluation of the druglikeness of compounds.
# pharmacokinetic profile: Absorption, Distribution, Metabolism and Excretion (ADME)

# Molecular weight < 500 Dalton
# Octanol-water partition coefficient (LogP) < 5
# Hydrogen bond donors < 5
# Hydrogen bond acceptors < 10

In [ ]:
# Import libraries
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [ ]:
# Calculate descriptors
# Inspired by: https://codeocean.com/explore/capsules?query=tag:data-curation

def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [ ]:
bioactivity_data_smiles = bioactivity_data_smiles[bioactivity_data_smiles.canonical_smiles.notna()]

In [ ]:
bioactivity_data_smiles_lipinski = lipinski(bioactivity_data_smiles.canonical_smiles)
bioactivity_data_smiles_lipinski

ArgumentError: ignored